#### Transport

计算采样点所需的参数(先验密度，drift，score)及训练loss.LoRA训练会用到这部分,后文的`model`即是`HYVideoDiffusionTransformer`，使用DeepSpeed框架训练.

- `ModelType`: 可选`NOISE`, `SCORE`, `VELOCITY`，控制get_drift()和get_score()的计算方法，指定预测目标。默认为`VELOCITY`.
- `WeightType`: 可选`VELOCITY`, `LIKELIHOOD`, `NONE`，控制loss的加权方式。默认为`NONE`.
- `SNRType`: 可选`UNIFORM`, `LOGNORM`，控制sample方法中时间$t$的分布(均匀或对数正态)，影响信噪比的分布。默认为`UNIFORM`.
- `PathType`: 可选`LINEAR`, `GVP`, `VP`，指定噪声和数据之间的插值路径。默认为`LINEAR`.
- `reverse`: 默认为`False`

实际使用`__init__.py`中的create_transport( )方法初始化其实例.`train_eps`和`sample_eps`用于早停(避免数值不稳定).

training_losses( )方法定义了DiT的训练损失,这里只按照默认的`VELOCITY`模式来说明。损失函数为$$\mathcal{L}(\theta)=\mathbb{E}_{x_0,x_1,t}\left[\left\|v_\theta(x_t,t)-x_t\right\|^2\right]$$
其中$t$的生成来自于self.sample( )方法, 依赖于`SNRType`,`video_shift`和`x1_batch`.`UNIFORM`会为每个$x_1$中每份图片生成一个$t\sim U(t_0,t_1)$,`LOGNORM`会生成$t\sim logit\mathcal{N}(0,1)$从$(0,1)$平移到$(t_0,t_1)$的结果，大致呈钟形，约50%的样本分布在$(0.355,0.645)$之间。这意味着前者训练中的噪声强度是均匀的,而后者会集中在中部。最终输出前，它们会再经$t = \frac{t}{shift - (shift - 1) * t}$缩小一点。

![LOGITNORM](./img/logitnormal.png)

In [ ]:
def training_losses(self, model, x1, model_kwargs=None, timestep=None, n_tokens=None,
                        i2v_mode=False, cond_latents=None, args=None):

        self.shift = self.video_shift
        if model_kwargs == None:
            model_kwargs = {}

        t, x0, x1 = self.sample(x1, n_tokens)
        if timestep is not None:
            t = th.ones_like(t) * timestep
        t, xt, ut = self.path_sampler.plan(t, x0, x1)  # 用t，x0，x1计算ut，xt。非reverse时，xt=mu_t=tx1+(1-t)x0是加噪结果
        input_t = self.get_model_t(t)  # 1000*t,数值

        if i2v_mode and args.i2v_condition_type == "latent_concat":
            if cond_latents is not None:
                x1_concat = cond_latents.repeat(1,1,x1.shape[2],1,1)
                x1_concat[:, :, 1:, :, :] = 0.0
            else:
                x1_concat = x1.cpu().clone().to(device=x1.device)
                x1_concat[:, :, 1:, :, :] = 0.0

            mask_concat = th.ones(x1.shape[0], 1, x1.shape[2], x1.shape[3], x1.shape[4]).to(device=x1.device)
            mask_concat[:, :, 1:, ...] = 0.0

            xt = th.concat([xt, x1_concat, mask_concat], dim=1)
        elif i2v_mode and args.i2v_condition_type == "token_replace":
            xt = th.concat([cond_latents, xt[:, :, 1:, :, :]], dim=2)  # 训练时，进入模型前，每个视频任务第一帧都被替换(加噪后的图->第一帧换回条件->进入网络)

        guidance_expand = (
            th.tensor(
                [args.embedded_cfg_scale] * x1.shape[0],
                dtype=th.float32,
                device=x1.device,
            ).to(PRECISION_TO_TYPE[args.precision])
            * 1000.0
            if args.embedded_cfg_scale is not None
            else None
        )  # None或(1000*cfg_scale),复制batch个
        model_kwargs["guidance"] = guidance_expand

        model_output = model(xt, input_t, **model_kwargs)['x']  # 进一次model这里应该自带一个循环

        if i2v_mode and args.i2v_condition_type == "token_replace":
            assert self.model_type == ModelType.VELOCITY, f"self.model_type: {self.model_type} must be ModelType.VELOCITY"
            model_output = model_output[:, :, 1:, :, :]  # 扔掉第一帧的单步去噪图
            ut = ut[:, :, 1:, :, :]  # 扔掉第一帧预测的ut，它是x1-x0(mut关于t的导数)
            
        terms = {}
        if self.model_type == ModelType.VELOCITY:
            terms["loss"] = mean_flat(((model_output - ut) ** 2))  # 每个维度求平均后求模.这里已经关于batch做了平均，看上去是整个batch塞进去的，可能训练框架有别的处理
            # 在启用token_replace的网络中, 第一帧不参与去噪循环,所以计算之前把u_t也去掉一帧,少算一帧
            if self.model_type == ModelType.NOISE:
                terms['loss'] = mean_flat(weight * ((model_output - x0) ** 2))
            else:
                terms['loss'] = mean_flat(weight * ((model_output * sigma_t + x0) ** 2))

        return model_output, terms

#### Sampler

基于Transport实例计算`drift`和`score`用于采样，并构建完整的采样器对象

$$\text{sde\_drift}(x, t) = v_\theta(x_t, t) + g(t)^2 \cdot \nabla_x \log p_t(x)$$

$$\nabla_x \log p_t(x)=\frac{\frac{\alpha_t}{\dot{\alpha}_t} v_\theta(x_t, t) - x_t}{\sigma_t^2 - \frac{\alpha_t}{\dot{\alpha}_t} \dot{\sigma}_t \sigma_t}$$

代码中`diffusion_fn`就是$g(t)$,默认为`SBDM`, `model(x,t)`就是$v_{\theta}(x_t,t)$.

采样时，先用transport.check_interval()生成起止时间，再用sde/ode求解器生成采样路径。sde的样本还需再经一步last_step_fn()去噪，作为列表的最后一个元素输出。

#### ICPlan

调度器基类，计算$\alpha_t, \sigma_t, d\alpha_t, drift, velocity, noise$等。还可在VPCPlan/GVPCPlan类中覆写$\alpha_t, \sigma_t, d\alpha_t, drift$的计算

#### sde/ode

求解器，负责潜变量x的更新循环

Euler采样器更新公式为$$x_{t+\Delta t} = x_t + f_\theta(x_t,t)\Delta t + \sqrt{2g(t)}\,\varepsilon\sqrt{\Delta t},\quad\varepsilon\sim\mathcal N(0,I)$$

Heun采样器更新公式为$$\begin{aligned}
\hat{x} &= x_t + \sqrt{2g(t)}\varepsilon \\
x_p &= \hat{x} + f_{\theta}(\hat{x}, t)\Delta t \\
x_{t+\Delta t} &= \hat{x} + \frac{\Delta t}{2} \left[ f_{\theta}(\hat{x}, t) + f_{\theta}(x_p, t + \Delta t) \right]
\end{aligned}$$

（暂定）对`velocity`模式，模型直接学习从噪声(t=0)到数据(t=1)的速度场，在采样时，时间是增大的

#### DiT

[U-ViT](https://arxiv.org/abs/2209.12152)和[DiT](https://arxiv.org/abs/2212.09748)均尝试用多个Transformer块替代U-Net中的卷积块预测噪声，且保留了浅层网络到深层网络的Skip Connection。两篇论文中，前者将 $t$ 和 $c$ 的嵌入向量作为两个附加的词元添加到输入序列中，这称为“上下文条件生成”；后者比较上下文条件生成、交叉注意力(Gflops开销增加显著，约15%)、adaLN(自适应层归一化)后选择了adaLN的方式注入条件。

![三种条件注入方式的比较](./img/hydit_comp.png)

AdaLN实际就是用小型的MLP学习对层归一化结果的仿射变换，使得条件去噪效果最好。其中$\gamma(c)$和$\beta(c)$是待学习的仿射变换系数。adaLN对数据的操作可表示为$$y=\gamma(c)\cdot\frac{x-\mu}{\sigma}+\beta(c)$$其中均值、方差的计算在最后一维进行。

![AdaLN图示](./img/hydit_adaln.png)

#### 部分问题的回答

`layer_norm`, `group_norm`, `spatial_norm`, `batch_norm`分别是什么？有什么应用场景？使用的原因分别是什么？
1. Batch Norm

只以$x_i$形如$[B,C,H,W]$为例.设有mini-batch的一组输入$\{x_i\}_{i=1}^{\mathcal{B}}$,`batch_norm`会在**通道**维度为该批数据求$C$组均值和方差来归一化，其中每个元素为$$\begin{gathered}
\hat{x}_{ichw}=\gamma_{c}\cdot \frac{x_{ichw}-\mu_{c}}{\sqrt{\sigma_{c}^{2}+\epsilon}}+\beta_{c}\\
\mu_c=\frac{1}{B\times H\times W}\sum_{i=1}^B\sum_{h=1}^H\sum_{w=1}^Wx_{ichw}\\
\sigma_{c}^{2}=\frac{1}{B\times H\times W}\sum_{i=1}^{B}\sum_{h=1}^{H}\sum_{w=1}^{W}(x_{ichw}-\mu_{c})^{2}
\end{gathered}$$
系数被用于“恢复该层的表征能力”(暂时不解)，详见[Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)

2. Layer Norm

以**单个样本**为单位求均值和方差，在每个样本内部把$C*H*W$的数据归一化.

3. Group Norm

以**单个样本的一组通道**为单位求均值和方差，在每个样本的通道组$C_g$内部把$C_g*H*W$的数据归一化.

4. Spatial Norm

来自`diffusers.models.attention_processor`实际是一个组归一，仿射系数用卷积从`zq`中学习

In [ ]:
class SpatialNorm(nn.Module):
    """
    Spatially conditioned normalization as defined in https://huggingface.co/papers/2209.09002.

    Args:
        f_channels (`int`):
            The number of channels for input to group normalization layer, and output of the spatial norm layer.
        zq_channels (`int`):
            The number of channels for the quantized vector as described in the paper.
    """

    def __init__(
        self,
        f_channels: int,
        zq_channels: int,
    ):
        super().__init__()
        self.norm_layer = nn.GroupNorm(num_channels=f_channels, num_groups=32, eps=1e-6, affine=True)
        self.conv_y = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)
        self.conv_b = nn.Conv2d(zq_channels, f_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, f: torch.Tensor, zq: torch.Tensor) -> torch.Tensor:
        f_size = f.shape[-2:]
        zq = F.interpolate(zq, size=f_size, mode="nearest")
        norm_f = self.norm_layer(f)
        new_f = norm_f * self.conv_y(zq) + self.conv_b(zq)
        return new_f

#实际调用
self.norm1 = SpatialNorm(in_channels, temb_channels)
hidden_states = input_tensor
if self.time_embedding_norm == spatial":
    hidden_states = self.norm1(hidden_states, temb)

在VAE的因果卷积块中，`resnet_time_scale_shift`被置为`default`, 即时间步会通过mlp后与`img`相加，一起经过GroupNorm。如置为SpatialNorm则会通过1*1卷积用时间学习归一化参数。

在DiT中，主要是`img`的`pre_norm`(层归一), `qk_norm`(RMS层归一，舍去中心化，分母也是非中心的。它只对原始分布做缩放，而不平移), 以及调制时的adaLN和final层的层归一。